In [1]:
import logging
import pandas as pd
from math import sqrt, log2
import numpy as np
import copy
import re
from numpy.linalg import norm
from itertools import product, combinations

try:
    import cPickle as pickle
except ImportError:  # python 3.x
    import pickle

import os, shutil
from random import randint
import datetime
import math


In [2]:
logging.basicConfig(format="%(asctime)s - %(thread)s - %(levelname)s - %(message)s")
logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [3]:
def doThisPathExist(path):
    return os.path.exists(path)

def createPath(path):
    if not doThisPathExist(path):
        try:
            os.makedirs(path)
        except RuntimeError:
            return False
        
        return True
    
def getRandomStringasId(i=None):
    if i is None:   
        utc_now = datetime.datetime.now()
        randomValueAsId = math.ceil(float((utc_now - datetime.datetime(1970, 1, 1)).total_seconds())) + randint(0, 9999)
        return str(randomValueAsId)
    else:
        return "DataSet_" + str(i)

def clearTheDirectory(path):
    if not doThisPathExist(path):
        raise FileNotFoundError("Error! The passed path do not exist!")
    else:
        for filename in os.listdir(path):
            file_path = os.path.join(path, filename)
            try:
                if os.path.isfile(file_path) or os.path.islink(file_path):
                    os.unlink(file_path)
                elif os.path.isdir(file_path):
                    shutil.rmtree(file_path)
                    
            except Exception as e:
                raise RuntimeError ('Failed to delete %s. Reason: %s' % (file_path, e))
        
        return True


In [4]:
def readPicklefile(path):
    with open(path, 'rb') as fp:
        data = pickle.load(fp)
    return data
    
def writePickleFile(path, data):
    with open(path, 'wb') as fp:
        pickle.dump(data, fp, protocol=pickle.HIGHEST_PROTOCOL)
    return True

In [5]:
class BuildTree_Iterative(object):
    def __init__(self, min_split, datasetDirectory='./', categoricalAttributes = []):
        
        self.min_split = min_split
        self.root = None
        self.maxNodeId = 0
        self.datasetDirectory = datasetDirectory
        self.categoricalAttributes = categoricalAttributes
#         tmpFilesPath = self.datasetDirectory + "TmpFilesPath/"
#         if not doThisPathExist(tmpFilesPath):
#             createPath(tmpFilesPath)
#         self.tmpFilesPath = tmpFilesPath
        self.cutCreator = InfoGainCutFactory(self.min_split)
        self.datasetSplitter = DatasetSplitter()
        
    def build(self, datasetPhysicalAddress):
        self._build_tree_iterative(datasetPhysicalAddress)
        return self.root

    def _build_tree_iterative(self, datasetPhysicalAddress):
        print ("XXXXXXXXXX----------XXXXXXXXXX----------XXXXXXXXXX----------XXXXXXXXXX----------XXXXXXXXXX")
        stack = list()
        parent = None
        datasetId = getRandomStringasId(0)
        depth = 0
        dataset = readPicklefile(datasetPhysicalAddress)
        DataSet_ParentChildMap = {}
        DataSet_ChildParentMap = {}
        CLNodes_IdToObjectMap = {}
        CLNodeIdToDataSetIdMap = {}
        DataSetIdToCLNodeIdMap = {}

        while True:
            if parent is not None:
                print ("At previous level({}), the cut happened on Attribute:{}, Value: {}".
                                       format(parent.depth, parent.attribute, parent.cutValue))
            else:
                print ("This is the starting point!")
            
#             print ("Underlying Dataset: \n{}".format(dataset.__str__()))
            
            bestCut = self._findBestCut(dataset)
            cutValue = None if bestCut is None else bestCut.value
            attribute = "" if bestCut is None else bestCut.attribute
            dataset.calculate_limits()
            attr_names = dataset.attr_names
            attr_idx = dataset.attr_idx
            max_values = dataset.max_values
            min_values = dataset.min_values
            length = dataset.getLength()
            curNodeId = self.maxNodeId 
            dt_node = CLNode(curNodeId, datasetPhysicalAddress, datasetId, parent, attribute, depth, cutValue,
                             attr_names, attr_idx, max_values, min_values, length)

            CLNodes_IdToObjectMap[curNodeId] = dt_node
            CLNodeIdToDataSetIdMap[curNodeId] = datasetId
            DataSetIdToCLNodeIdMap[datasetId] = curNodeId
            self.maxNodeId += 1

            if parent: parent.addChildNode(dt_node)
            if self._isRootNode(depth): self.root = dt_node

            tapIntoRightDS = True

            if bestCut is not None:
                lhs_dataset, rhs_dataset = self._splitDatasetUsingBestCut(dataset, bestCut, adjust=True)  # Changes Made
                parentDatasetId = datasetId
                datasetId = getRandomStringasId(curNodeId)
                L_datasetId = str(datasetId) + "_L"
#                 L_datasetPhysicalAddress = self.datasetDirectory + str(L_datasetId) + ".pickle"
                lhs_dataset.calculate_limits()
#                 writePickleFile(L_datasetPhysicalAddress, lhs_dataset)  #  May be remove this.

                R_datasetId = str(datasetId) + "_R"
                R_datasetPhysicalAddress = self.datasetDirectory + str(R_datasetId) + ".pickle"
                rhs_dataset.calculate_limits()
                writePickleFile(R_datasetPhysicalAddress, rhs_dataset)

                DataSet_ParentChildMap[parentDatasetId] = (L_datasetId, R_datasetId)
                DataSet_ChildParentMap[L_datasetId] = parentDatasetId
                DataSet_ChildParentMap[R_datasetId] = parentDatasetId

                stack.append(R_datasetId)

                if lhs_dataset.getLength() > self.min_split:
                    dataset = lhs_dataset
                    parent = dt_node
                    datasetId = L_datasetId
                    depth = depth + 1
                    tapIntoRightDS = False
                    datasetPhysicalAddress = None

                else:
                    L_datasetPhysicalAddress = self.datasetDirectory + str(L_datasetId) + ".pickle"
                    writePickleFile(L_datasetPhysicalAddress, lhs_dataset)
                    
                    attr_names = lhs_dataset.attr_names
                    attr_idx = lhs_dataset.attr_idx
                    max_values = lhs_dataset.max_values
                    min_values = lhs_dataset.min_values
                    length = lhs_dataset.getLength()
                    curNodeId = self.maxNodeId
                    cNode = CLNode(curNodeId, L_datasetPhysicalAddress, L_datasetId, dt_node, None, depth + 1, None,
                                   attr_names, attr_idx, max_values, min_values, length)
                    dt_node.addChildNode(cNode)
                    CLNodes_IdToObjectMap[curNodeId] = cNode
                    CLNodeIdToDataSetIdMap[curNodeId] = L_datasetId
                    DataSetIdToCLNodeIdMap[L_datasetId] = curNodeId
                    self.maxNodeId += 1
            else:
                
                datasetPhysicalAddress = self.datasetDirectory + str(datasetId) + ".pickle"
                writePickleFile(data = dataset, path = datasetPhysicalAddress)
                dt_node.modifyUnderlyingDataSet(datasetId=datasetId, datasetPhysicalAddress=datasetPhysicalAddress)

            if tapIntoRightDS:
                if len(stack) > 0:
                    R_datasetId = stack.pop()
                    datasetId = R_datasetId
                    ParentDataSetId = DataSet_ChildParentMap[R_datasetId]
                    ParentNodeId = DataSetIdToCLNodeIdMap[ParentDataSetId]
                    parent = CLNodes_IdToObjectMap[ParentNodeId]
                    depth = parent.getDepth() + 1
                    R_datasetPhysicalAddress = self.datasetDirectory + R_datasetId + ".pickle"
                    datasetPhysicalAddress = R_datasetPhysicalAddress
                    dataset = readPicklefile(R_datasetPhysicalAddress)
                else:
                    break

    def _isRootNode(self, depth):
        if depth == 0 and self.root is None: return True

    def _splitDatasetUsingBestCut(self, dataset, bestCut, adjust):  # Changes Made
        dataset.sort(bestCut.attribute)
        idx = dataset.getInstanceIndex(bestCut.inst_id)
        try:
            idx = idx[-1]
        except:
            pass

        lhs_set, rhs_set = self.datasetSplitter.split(dataset, bestCut.attribute, bestCut.value, idx + 1,
                                                      adjust)  # Changes Made
        lhs_set.calculate_limits()
        rhs_set.calculate_limits()

        return lhs_set, rhs_set

    def _findBestCut(self, dataset):
        dataset.calculate_limits()
        bestCut = None
        
#         print("Trying to find best cut among the following attributes: {}".format(list(dataset.attr_names)))
        for attribute in dataset.attr_names:
            dataset.sort(attribute)  # Sorting the dataset based on a attribute value at a time.
            di_cut1 = self._calcCut1(dataset, attribute)  # Best cut on current attribute based on "Information Gain"
            if di_cut1 is None:  # Ignore dimension go to the next dimension.
                continue
            if len(self.categoricalAttributes)>0 and attribute in self.categoricalAttributes:
                bestCut = self._selectLowerDensityCut(di_cut1, bestCut)
                continue
#             print("Attribute: {}, Cut-1 Created at {}".format(di_cut1.attribute, di_cut1.value))
            di_cut2 = self._calcCut2(di_cut1)  # Acquiring a second cut on the current attribute at Low Density Region.
            if di_cut2 is None:  # If there is no di_cut2 there, di_cut1 is the best cut on this dimension.
                bestCut = self._selectLowerDensityCut(di_cut1, bestCut)  # Keeping Track of best cut incrementally.
                continue  # We are done with this dimension, skip the rest and go to the next dimension.
#             print("Attribute: {}, Cut-2 Created at {}".format(di_cut2.attribute, di_cut2.value))
            di_cut3 = self._calcCut3(di_cut1,
                                     di_cut2)  # Acquire a cut in the space which is in between di_cut1 and di_cut2.
            if di_cut3 is None:  # if di_cut3 is no better than di_cut2 in terms of relative density the di_cut2 is the best cut on this dimesion.
                bestCut = self._selectLowerDensityCut(di_cut2, bestCut)  # Keeping Track of best cut incrementally.
            else:  # If there is a di_cut3 that means there is a lower density region and di_cut3 is the best cut on this dimension.
#                 print("Attribute: {}, Cut-3 Created at {}".format(di_cut3.attribute, di_cut3.value))
                bestCut = self._selectLowerDensityCut(di_cut3, bestCut)  # Keeping Track of best cut incrementally.

#             print("Best Cut So Far on Attribute: {}, at {}".format(bestCut.attribute, bestCut.value))
#         if bestCut is not None:
#             print("Best Cut Happended on Attribute: {}, at {}".format(bestCut.attribute, bestCut.value))
#         else:
#             print("No cut is possible at on the underlying dataset. {}".format(dataset.instance_view))

        return bestCut  # Returning the best cut at this level and the local dataset, which is a subset of the entire dataset.

    def _calcCut1(self, dataset, attribute):
        return self.cutCreator.cut(dataset,
                                   attribute)  # Acquire the first cut on the dimension based on Information Gain

    def _calcCut2(self, di_cut1):
        lower_density_set, _ = di_cut1.getLowerDensityRegion()  # Acquire the lower realtive density region among the regions created by di_cut1
#         print("For Cut1, the lower density set is: \n {}".format(lower_density_set.__str__()))
        return self.cutCreator.cut(lower_density_set,
                                   di_cut1.attribute)  # Acquire the second cut on the dimension in the Lower Density Region based on Information Gain.

    def _calcCut3(self, di_cut1, di_cut2):
        adjacentRegion, otherRegion = di_cut2.getAdjacentRegion(di_cut1.value,
                                                   di_cut1.attribute)  # Acquire at which region the value used to acquire the di_cut1 falls in as the result of di_cut2.

#         print("For Cut2, the Adjacent Region is: \n{}".format(adjacentRegion.__str__()))
#         print("For Cut2, the Other Region is: \n{}".format(otherRegion.__str__()))
        di_cut3 = None
#         print("Relative Density of Adjacent Region: {}".format(_relative_density(adjacentRegion)))
#         print("Relative Density of Other Region: {}".format(_relative_density(otherRegion)))

        if _relative_density(adjacentRegion) <= _relative_density(
                otherRegion):  # Compare relative densities among two regions as the result of di_cut2.
#             print("Relative density of the adjacent region is lower than/equal to relative density of the other region")
            lower_density_set, _ = di_cut2.getLowerDensityRegion()  # Acquire the lower realtive density region among the regions created by di_cut2.
            
#             print("For Cut2, the lower density set is: \n{}".format(lower_density_set.__str__()))
            di_cut3 = self.cutCreator.cut(lower_density_set,
                                          di_cut2.attribute)  # Acquire the third cut on the dimension in the Lower Density Region based on Information Gain.
        return di_cut3

    def _selectLowerDensityCut(self, cut1, cut2):
        if cut1 is None: return cut2
        if cut2 is None: return cut1
        rd1 = cut1.getRelativeDensityOfLowerDensityRegion()  # Acquire the lower density region by cut1
        rd2 = cut2.getRelativeDensityOfLowerDensityRegion()  # Acquire the lower density region by cut2
#         print("Relative Density of cut on Attribute: {}, at Value: {}, is {}".format(cut1.attribute, cut1.value, rd1))
#         print("Relative Density of cut on Attribute: {}, at Value: {}, is {}".format(cut2.attribute, cut2.value, rd2))
        if rd1 < rd2:
#             print("The lowest density cut is cut1")
            return cut1  # Return the cut which has the lower density region with the lowest relative density.
        else:
#             print("The lowest density cut is cut2")
            return cut2


## Prior Knowledge Required to Understand The Concepts Explained Here Better:

#### Entropy And InformationGain (Pages:1-4):
https://www.math.unipd.it/~aiolli/corsi/0708/IR/Lez12.pdf

#### More Detail on Entropy And InformationGain:
https://www.cs.cmu.edu/~mgormley/courses/10601-s17/slides/lecture27-infotheory.pdf

#### Supervised Learning, Decision Tree Algorithm:

#### Decision Tree 1: how it works:
https://www.youtube.com/watchv=eKD5gxPPeY0&list=PLBv09BD7ez_4temBw7vLA19p3tdQH6FYO&index=1

#### ID3 algorithm: how it works:
https://www.youtube.com/watch?v=_XhOdSLlE5c&list=PLBv09BD7ez_4temBw7vLA19p3tdQH6FYO&index=2

#### Decision Tree 3: which attribute to split on?:
https://www.youtube.com/watch?v=AmCV4g7_-QM&list=PLBv09BD7ez_4temBw7vLA19p3tdQH6FYO&index=3

#### Decision Tree 4: Information Gain:
https://www.youtube.com/watch?v=nodQ2s0CUbI&list=PLBv09BD7ez_4temBw7vLA19p3tdQH6FYO&index=4

#### Clustering Through/Via Decision Tree Construction:
###### Paper(Shorter Version):
     http://ftp.cse.buffalo.edu/users/azhang/disc/disc01/cd1/out/papers/cikm/p20.pdf
###### Paper(More Detailed Version):
     http://web.cs.ucla.edu/~wwc/course/cs245a/CLTrees.pdf

#### Paper Presentation Video (There are perhaps many but this one I find more clean than others):
https://www.youtube.com/watch?v=9nisRs_vkGo&list=PL5dMgjRyXQMD4jqNet7A-2IDOFj4whfDM&index=2&t=780s
[Atleast watch till 11:30.]


In [6]:
def _relative_density(dataset):
    return float(dataset.getLength())/dataset.nr_virtual_points


In [7]:
class Data:

    '''DataFrame Data'''
    def __init__(self, instance_values, types):

        self.instance_values = instance_values
        self.attr_types = types
        self.attr_idx = dict()
        self.attr_names = list()
        self.totalNumberofDataPoints = None
        self._init_attr_names()
        self._init_max_min()  # Could replace with self.calculate_limits()

        self.nr_virtual_points = len(self.instance_values)
        self.nr_total_instances = 2*self.nr_virtual_points

    def _init_max_min(self):

        if len(self.instance_values) > 1:
            self.instance_view = self.instance_values.view(dtype=float).reshape(len(self.instance_values),-1)
            self.max_values = np.amax(self.instance_view, 0)
            self.min_values = np.amin(self.instance_view, 0)
        else:
            self.instance_view = self.instance_values.view(dtype=float)
            self.max_values = copy.copy(self.instance_view)
            self.min_values = copy.copy(self.instance_view)

    def _init_attr_names(self):

        for i, attr in enumerate(self.attr_types):
            if i < 1:
                continue
            attr_name, attr_type = attr
            self.attr_idx[attr_name] = i
            self.attr_names.append(attr_name)

    def __str__(self):

        s = 'Data: ' + str(len(self.instance_values)) + "\n"
        s += str(self.attr_names) + "\n"
        s += " Max :" + str(self.max_values)+ "\n"
        s += " Min :" + str(self.min_values)+ "\n"
        s += '\n--------\n'
        return s

    def calculate_limits(self):
        self._init_max_min()

    def sort(self, attribute):

        self.instance_values = np.sort(self.instance_values, order=attribute)
        self.instance_view = self.instance_values.view(dtype=float).reshape(len(self.instance_values), -1)

    def getLength(self):
        if self.totalNumberofDataPoints is None:
            self.totalNumberofDataPoints = len(self.instance_values)
            
        return self.totalNumberofDataPoints

    def getInstances(self, attribute):

        idx = self.attr_idx[attribute]
        if self.getLength() > 1:
            return self.instance_view[:, idx]
        elif self.getLength() == 1:
            return [self.instance_view[0, idx]]
        else:
            return []

    def getInstanceIndex(self, id):

        if self.getLength() > 1:
            idx = np.argwhere(self.instance_view[:,0] == id)
            return idx[0]
        elif self.getLength() == 1 and id == self.instance_view[0]:
            return 0
        else:
            return None

    def getId(self, idx):

        if self.getLength() > 1:
            return self.instance_view[idx][0]
        elif self.getLength() == 1:
            return self.instance_view[0]
        else:
            return -1

    def get_max(self, attribute):

        idx = self.attr_idx[attribute]
        return self.max_values[idx]

    def get_min(self, attribute):

        idx = self.attr_idx[attribute]
        return self.min_values[idx]

    def set_max(self, attribute, value):

        if len(self.max_values) > 0:
            idx = self.attr_idx[attribute]
            self.max_values[idx] = value

    def set_min(self, attribute, value):

        if len(self.min_values) > 0:
            idx = self.attr_idx[attribute]
            self.min_values[idx] = value

class DataFrameReader:

    def __init__(self, df):
        self.df = df
        pass

    def read(self):

        types = self._read_columnNames()
        output = self._read_instances()

        output = np.array(output, dtype=types)
        data = Data(output, types)

        logger.info("read " + str(len(self.df)))
        logger.info("attribute names: {}".format(data.attr_types))

        return data

    def _read_columnNames(self):

        dtype = list()
        dtype.append(("id", float))
        for col in list(self.df.columns):
            dtype.append((col, float))

        return dtype

    def _read_instances(self):

        output = list()
        count = 0.0
        for index, row in self.df.iterrows():
            a = tuple(row.values)
            a = tuple([count]) + a
            output.append(a)
            count += 1.0
        return output

class DataSetSummary:

    def __init__(self, dataSetId, datasetPhysicalAddress, attr_names, attr_idx, max_values, min_values, length):

        self.dataSetId = dataSetId
        self.datasetPhysicalAddress = datasetPhysicalAddress
        self.attr_names = attr_names
        self.attr_idx = attr_idx
        self.max_values = max_values
        self.min_values = min_values
        self.length = length
        return

    def get_max(self, attribute):
        idx = self.attr_idx[attribute]
        return self.max_values[idx]

    def get_min(self, attribute):
        idx = self.attr_idx[attribute]
        return self.min_values[idx]

    def getLength(self):
        return self.length

    def getId(self):
        return self.dataSetId

    def getPhysicalAddress(self):
        return self.datasetPhysicalAddress

    def getAttributes(self):
        return self.attr_names

    def getAttrIndexes(self):
        return self.attr_idx

    def __str__(self):
        s = "Data Set Id: {} ".format(self.getId())
        s = s + "Physical Address: {} ".format(self.getPhysicalAddress())
        s = s + "Total Number of Data Points: {} ".format(self.getLength())
        s = s + "Attributes: {} ".format(self.getAttributes())
        s = s + "Min Values: {} ".format(self.min_values)
        s = s + "Max Values: {}".format(self.max_values)
        return s


In [8]:
class DatasetSplitter:
    def __init__(self):
        pass
    
    def split(self, dataset, attribute, value, idx, adjust=False): # Changes Made
        
        l = dataset.instance_values[0: idx]
        r = dataset.instance_values[idx:]
        lhs_set = Data(l, dataset.attr_types)
        rhs_set = Data(r, dataset.attr_types)

        # Set the Lhs min and Max and Rhs Max. 
        lhs_set.calculate_limits()
        rhs_set.calculate_limits()
        
        self._splitNrVirtualPoints(dataset, attribute, value, lhs_set, rhs_set)
        if adjust: # Changes Made
#             print ("Before Adjusting The Number of Virtual Points:")
#             print ("\t LHS - Actual Points: {}, Virtual Points: {}, Total Number of Data Points: {}".format(\
#                                             lhs_set.getLength(), lhs_set.nr_virtual_points, lhs_set.nr_total_instances))
#             print ("\t RHS - Actual Points: {}, Virtual Points: {}, Total Number of Data Points: {}".format(\
#                                             rhs_set.getLength(), rhs_set.nr_virtual_points, rhs_set.nr_total_instances))
            self._updateVirtualPoints(lhs_set)
            self._updateVirtualPoints(rhs_set)
            
#             print ("After Adjusting The Number of Virtual Points:")
#             print ("\t LHS - Actual Points: {}, Virtual Points: {}, Total Number of Data Points: {}".format(\
#                                             lhs_set.getLength(), lhs_set.nr_virtual_points, lhs_set.nr_total_instances))
#             print ("\t RHS - Actual Points: {}, Virtual Points: {}, Total Number of Data Points: {}".format(\
#                                             rhs_set.getLength(), rhs_set.nr_virtual_points, rhs_set.nr_total_instances))
        else:
            lhs_set.nr_total_instances = lhs_set.getLength() + lhs_set.nr_virtual_points
            rhs_set.nr_total_instances = rhs_set.getLength() + rhs_set.nr_virtual_points
            
        return lhs_set, rhs_set
        
    def _splitNrVirtualPoints(self, dataset, attribute, value, in_set, out_set):
        minV = dataset.get_min(attribute)
        maxV = dataset.get_max(attribute)
        
        in_set.nr_virtual_points = max(int(abs(dataset.nr_virtual_points*((value-minV+1)/(maxV-minV+1)))),1) # Changes Made
        out_set.nr_virtual_points = dataset.nr_virtual_points - in_set.nr_virtual_points
#         if in_set.nr_virtual_points ==0:
#             print ("Original Virtual Points: {}, Out Set Virtual Points: {},\n abs(dataset.nr_virtual_points*((value-minV+1)/(maxV-minV+1))): {}".format(dataset.nr_virtual_points,out_set.nr_virtual_points, abs(dataset.nr_virtual_points*((value-minV+1)/(maxV-minV+1))) ))
        if out_set.nr_virtual_points < 0:
            self.raiseUndefinedNumberOfPoints()
            
        return
    
    def _updateVirtualPoints(self, data_set):            
        nr_points_in_set = data_set.getLength()
        data_set.nr_virtual_points = self._calcNumberOfPointsToAdd(nr_points_in_set, data_set.nr_virtual_points)
        data_set.nr_total_instances = nr_points_in_set + data_set.nr_virtual_points
        return
    
    def _calcNumberOfPointsToAdd(self, nr_points_in_node, nr_points_inherited):    
        if nr_points_inherited < nr_points_in_node:
            nr_points = nr_points_in_node
        else:
            nr_points = nr_points_inherited
        return nr_points
    
    def raiseUndefinedNumberOfPoints(self):
        raise DatasetSplitter.UndefinedNumberOfPoints()
    class UndefinedNumberOfPoints(Exception):
        pass
    

In [9]:
class Cut:
    def __init__(self, attribute, value, inst_id, lhsset, rhsset):
        self.attribute = attribute
#         print ("Cut Value: {}".format(value))
        self.value = value
        self.inst_id = inst_id
        self.lhsset = lhsset
        self.rhsset = rhsset
        self.lowerDensityRegion = None
        self.lowerDensity = None

#         runningId = str(getRandomStringasId())
#         lhsFileName = runningId + '_L_' + attribute + '_' + str(value) +'.pickle'
#         lhsPath = tmpFilesPath + lhsFileName
#         rhsFileName = runningId + '_R_' + attribute + '_' + str(value) +'.pickle'
#         rhsPath = tmpFilesPath + rhsFileName
#         if doThisPathExist(lhsPath):
#             raise FileExistsError ("Error! Temporary file named '{}' already exists at {}!"\
#                                    .format(lhsFileName, tmpFilesPath))
#         if doThisPathExist(rhsPath):
#             raise FileExistsError ("Error! Temporary file named '{}' already exists at {}!"\
#                                    .format(rhsFileName, tmpFilesPath)) 
#         writePickleFile(path=lhsPath, data = lhsset)
#         writePickleFile(path=rhsPath, data = rhsset)
#         self.rhsPath = rhsPath
#         self.lhsPath = lhsPath

    def __str__(self):
        s = 'Cut: ' + self.attribute + "\n"
        s += 'At: ' + str(self.value)
        return s
        
    def getAdjacentRegion(self, value, attribute):
        def getMinimumDistanceFromValue(dataset, attribute, value):
            distance1 = abs(dataset.get_max(attribute) - value)
            distance2 = abs(dataset.get_min(attribute) - value)
            return min(distance1, distance2)
#         rhs_set = readPicklefile(self.rhsPath)
#         rhs_set.calculate_limits()
        
#         lhs_set = readPicklefile(self.lhsPath)
#         lhs_set.calculate_limits()
        rhs_distance = getMinimumDistanceFromValue(self.rhsset, attribute, value)
        lhs_distance = getMinimumDistanceFromValue(self.lhsset, attribute, value)
#         print ("*****************")
#         print (rhs_set)
#         print (rhs_distance)
#         print (lhs_set)
#         print (lhs_distance)
#         print (value)
#         print ("*****************")
        if lhs_distance < rhs_distance:
            return self.lhsset, self.rhsset
        else:
            return self.rhsset, self.lhsset
    
    def getRelativeDensityOfLowerDensityRegion(self):
        if self.lowerDensity is None:
            _, lowerDensity = self.getLowerDensityRegion()
            self.lowerDensity = lowerDensity
            
        return self.lowerDensity

    def getLowerDensityRegion(self):
        if self.lowerDensityRegion is None:
        
            if self.lhsset is None or self.rhsset is None:
                self.raiseNoRegionsDefined()

            rdL = _relative_density(self.lhsset)
            rdR = _relative_density(self.rhsset)
            if rdL > rdR:
                self.lowerDensityRegion = self.rhsset
                self.lowerDensity = rdR
            else:
                self.lowerDensityRegion = self.lhsset
                self.lowerDensity = rdL
        
        return self.lowerDensityRegion, self.lowerDensity
    
    def raiseNoRegionsDefined(self):
        raise Cut.NoRegionsDefined()
    class NoRegionsDefined(Exception):
        pass  

In [10]:
class InfoGainCutFactory:
    def __init__(self, min_split):
        self.min_split = min_split
        self.datasetSplitter = DatasetSplitter()
        
    def cut(self, dataset, attribute):
        dataset.sort(attribute)
        di_cut = None
        di_cut_setting = {}
        max_info_gain = -1          
        instances = dataset.getInstances(attribute)
        nunique = np.unique(instances)
        for value in nunique:
#             print ("Current Attribute: {}, Current Value: {}".format(attribute, value))
            i = np.argwhere(instances == value)
            i = i[-1] 
            i = i[0]
                
            if self._hasRectangle(dataset, attribute, value):
                
                lhs_set, rhs_set = self.datasetSplitter.split(dataset, attribute, value, i+1)                
                ig = self._info_gain(dataset, lhs_set, rhs_set)
                if ig > max_info_gain:
#                     print("Max Info. Gain Modified! Att.: {}, Val: {}, new Max: {}".format(attribute, value, ig))
                    max_info_gain = ig
                    di_cut_setting['attribute'] = attribute
                    di_cut_setting['value'] = value
                    di_cut_setting['id'] = dataset.getId(i)
                    di_cut_setting['index'] = i
        
#         print("di_cut_setting")
#         print (di_cut_setting)            
        if max_info_gain > -1:
            lset, rset = self.datasetSplitter.split(dataset, attribute,
                                                          di_cut_setting['value'],
                                                          di_cut_setting['index']+1)
            di_cut = Cut(attribute, di_cut_setting['value'], dataset.getId(di_cut_setting['index']), lset, rset)
#         print(di_cut)
        
        return di_cut
    
    def _hasRectangle(self, dataset, attribute, value):
        if dataset.get_max(attribute) == dataset.get_min(attribute): 
            return False
        else:
            if dataset.get_max(attribute) == value:
                return False
            else:
                return True

    def _info_gain(self, dataset, lhs_set, rhs_set):
#         print ("XXXXXXXXXX----------XXXXXXXXXX----------XXXXXXXXXX----------XXXXXXXXXX----------XXXXXXXXXX")
        #if (lhs_set.nr_total_instances < self.min_split) or (rhs_set.nr_total_instances < self.min_split):
        if (lhs_set.nr_total_instances < 1) or (rhs_set.nr_total_instances < 1):
            return -1
    
        ratio_instances_lhs = (float(lhs_set.nr_total_instances)/dataset.nr_total_instances)
        ratio_instances_rhs = (float(rhs_set.nr_total_instances)/dataset.nr_total_instances)
#         print ("LHS Ratio: {}, RHS Ratio: {}".format(ratio_instances_lhs, ratio_instances_rhs))
#         print("Calculating Entropy Within LHS:")
        entropyLHS = self._calc_entropy(lhs_set)
#         print("Calculating Entropy Within RHS:")
        entropyRHS = self._calc_entropy(rhs_set)
        entropy2 = ratio_instances_lhs*entropyLHS + ratio_instances_rhs*entropyRHS
#         print("\nCalculating Entropy of the Whole Dataset at Current Situation:")
        entropy1 = self._calc_entropy(dataset)
#         print ("\nInformation-Gain if this cut is made: EntropyDataSet - (Weighted Sum of LHS and RHS Entropy) = {}".format(entropy1 - entropy2))
#         print ("XXXXXXXXXX----------XXXXXXXXXX----------XXXXXXXXXX----------XXXXXXXXXX----------XXXXXXXXXX")
        return (entropy1 - entropy2)

    def _calc_entropy(self, dataset):
        # Changes Made
        nr_existing_instances = dataset.getLength()
        total = nr_existing_instances + dataset.nr_virtual_points
        terms = list()
#         print("Calculating actual data-point's contribution in the Entropy of this subset of data!")
#         print("Equation: - (actual-datapoints ÷ total-datapoints) * log2(actual-datapoints ÷ total-datapoints)")
#         print ("- ({} ÷ {}) * log2({} ÷ {})".format(nr_existing_instances, total, nr_existing_instances, total))
        actualDataPointEntropyContribution = (float(nr_existing_instances)/float(total))*(np.log2(float(nr_existing_instances)/float(total)))
#         print ("Result: -1 * {}".format(actualDataPointEntropyContribution))
        terms.append(actualDataPointEntropyContribution)    
#         print("Calculating virtual data-point's contribution in the Entropy of this subset of data!")
#         print("Equation: - (virtual-datapoints ÷ total-datapoints) * log2(virtual-datapoints ÷ total-datapoints)")
#         print ("- ({} ÷ {}) * log2({} ÷ {})".format(dataset.nr_virtual_points, total, dataset.nr_virtual_points, total))
        virtualPointEntropyContribution = (float(dataset.nr_virtual_points)/float(total))*(np.log2(float(dataset.nr_virtual_points)/float(total)))
#         print ("Result: -1 * {}".format(virtualPointEntropyContribution))
        terms.append(virtualPointEntropyContribution)                
        return sum(terms)*-1

In [11]:
class CLNode(object):
    # Added cut to it. So that It can be reused. Cut can be represented by value and attribute.
    def __init__(self, nodeId, datasetPhysicalAddress, datasetId, parent, attribute, depth, cutValue,
                 dsAttrNames, dsAttrIdx, maxValues, minValues, length):

        self.modifyUnderlyingDataSet(datasetId, datasetPhysicalAddress)
        self.datasetSummary = DataSetSummary(datasetId, datasetPhysicalAddress, dsAttrNames,
                                             dsAttrIdx, maxValues, minValues, length)
        self.parent = parent
        self.attribute = attribute
        self.cutValue = cutValue
        self.depth = depth
        self.children = list()
        self.can_prune = False
        self.includedInCluster = False
        self.clusterId = None
        self.nodeId = nodeId
        self.touchedNodes = None
        self.modifiedLength = None
        self.dataset = None

    def getDatasetPhysicalAddress(self):
        return self.datasetPhysicalAddress

    def getDatasetId(self):
        return self.datasetId

    def modifyUnderlyingDataSet(self, datasetId, datasetPhysicalAddress):
        if datasetPhysicalAddress is None:
            self._setDatasetId(datasetId)
            self._setDatasetPhysicalAddress(datasetPhysicalAddress)
            return True
        
        elif doThisPathExist(datasetPhysicalAddress):
            try:
                self._setDatasetId(datasetId)
                self._setDatasetPhysicalAddress(datasetPhysicalAddress)
                return True
            except RuntimeError:
                logger.error("Error! Unable to modify/assign the underlying dataset!")
                raise RuntimeError("Error! Unable to modify/assign the underlying dataset!")
                return False
        else:
            logger.error("The path, {}, doesn't exist or the program, doesn't have access to the path!". \
                         format(datasetPhysicalAddress))
            raise FileNotFoundError("The path, {}, doesn't exist or the program, doesn't have access to the path!". \
                                    format(datasetPhysicalAddress))
            return False

    def _setDatasetId(self, ii):
        self.datasetId = ii
        return True

    def _setDatasetPhysicalAddress(self, datasetPhysicalAddress):
        self.datasetPhysicalAddress = datasetPhysicalAddress
        return True

    def setPruneState(self, prune):
        self.can_prune = prune

    def isPrune(self):
        return self.can_prune

    def getRelativeDensity(self):
        dataset = readPicklefile(self.datasetPhysicalAddress)
        return _relative_density(dataset) * 100.0

    def getNrInstancesInNode(self):
        return self.datasetSummary.getLength()

    def addChildNode(self, node):
        self.children.append(node)

    def getChildNodes(self):
        return self.children

    def getLeft(self):
        if len(self.children) == 0:
            return None
        else:
            return self.children[0]

    def getRight(self):
        if len(self.children) < 2:
            return None
        else:
            return self.children[1]

    def getParent(self):
        return self.parent

    def isLeaf(self):
        if len(self.children) == 0:
            return True
        else:
            return False

    def getId(self):
        return self.nodeId

    def getTouchedNodes(self):
        return self.touchedNodes

    def addTouchedNodes(self, nodeId):
        if self.touchedNodes is None:
            self.touchedNodes = list()

        self.touchedNodes.append(nodeId)
        return

    def getDepth(self):
        return self.depth

    def setModifiedDataLength(self, modifiedLength):
        self.modifiedLength = modifiedLength
        return

    def getModifiedDataLength(self):
        return self.modifiedLength

    def getDatasetSummary(self):
        return self.datasetSummary

    def raiseAddNode(self):
        raise CLNode.AddNodeIlogical("hi")

    class AddNodeIlogical(Exception):
        pass

    def __str__(self):
        attr = list()
        vals = list()
        p = self.parent
        while p:
            attr.append(p.attribute)
            vals.append(p.cutValue)
            p = p.parent

        s = 'Node: ' + '\n'
        s += str(self.getNrInstancesInNode()) + ' instances, '
        s += ", " + str(int(self.getRelativeDensity())) + " relative density " + '\n'
        s += ", " + str(self.depth) + " depth " + '\n'
        s += "Cuts " + str(attr) + " At Values " + str(vals) + '\n'
        return s

    def str2(self):
        s = 'ID: {}'.format(self.nodeId)
        s += ' Inst: {} '.format(self.getNrInstancesInNode())
        s += "Depth: {}".format(self.getDepth())
        s += '' if self.clusterId is None else self.clusterId
        return s

In [12]:
class CLTree:
    def __init__(self, dataset, min_split=100, tmpFilesPath='./'):
        if dataset is None:
            self.raiseUndefinedDataset()
            
        self.min_split = min_split
        self.clusters = dict()
        self.nodes = dict()
        self.clustersStatistics = None
        self.root = None
        
        while True:
            tmpRunningId = getRandomStringasId()
            tmpStoringPath = tmpFilesPath + "CLTree/" + str(tmpRunningId) + "/"
            if doThisPathExist(tmpStoringPath):
                continue
            else:
                self.runningId = tmpRunningId
                createPath(tmpStoringPath)
                self.storageDirectory = tmpStoringPath
                break

        datasetDirectory = self.storageDirectory + "DataSet/"
        self.datasetDirectory = datasetDirectory
        if not doThisPathExist(self.datasetDirectory):
            createPath(self.datasetDirectory)
        dataset.calculate_limits()
        categoricalAttributes = []
        for attr in data.attr_names:
            instances = data.getInstances(attr)
            nunique = np.unique(instances)
            if len(nunique) < 3:
                categoricalAttributes.append(attr)
        self.categoricalAttributes = categoricalAttributes
        
        self.datasetPhysicalAddress = self.datasetDirectory + 'DataSet_' + str(self.runningId) +'.pickle'
        writePickleFile(path=self.datasetPhysicalAddress, data=dataset)

    def buildTree(self):
        dataset = self.getDataset()
        b = BuildTree_Iterative(self.min_split, self.datasetDirectory, self.categoricalAttributes)
        self.root = b.build(self.datasetPhysicalAddress)
        allNodes = self._collectAllNodes(self.root, dict())
        self.nodes = allNodes

    def getDataset(self):
        dataset = readPicklefile(self.datasetPhysicalAddress)
        return dataset
    
    def getRoot(self):
        return self.root

    def getAllClusters(self):
        return self.clusters

    def getAllNodes(self):
        return self.nodes

    def getClustersStatistics(self):
        return self.clustersStatistics

    def setClustersStatistics(self, result):
        self.clustersStatistics = result
        allClusters = self._collectAllClusters(self.root, dict())
        self.clusters = allClusters
        return

    def _collectAllClusters(self, node, allClusters):

        if node.includedInCluster:
            clusterId = node.clusterId
            if clusterId in allClusters.keys():
                val = allClusters[clusterId]
                val.append(node.nodeId)
                allClusters[clusterId] = val
            else:
                allClusters[clusterId] = [node.nodeId]

        else:
            children = node.getChildNodes()
            self._collectAllClusters(children[0], allClusters)
            self._collectAllClusters(children[1], allClusters)

        return allClusters

    def _collectAllNodes(self, node, allNodes):
        # In-order Traversal
        if node.isLeaf():
            allNodes[node.nodeId] = node
            nodeDataset = readPicklefile(node.datasetPhysicalAddress)
            nodeDataset.calculate_limits()
            node.dataset = nodeDataset
            return allNodes

        children = node.getChildNodes()
        allNodes = self._collectAllNodes(children[0], allNodes)
        allNodes[node.nodeId] = node
        allNodes = self._collectAllNodes(children[1], allNodes)
        return allNodes

    def raiseUndefinedDataset(self):
        raise CLTree.UndefinedDataset()

    class UndefinedDataset(Exception):
        pass

    def raiseUndefinedTree(self):
        raise CLTree.UndefinedTree()

    class UndefinedTree(Exception):
        pass

In [13]:
df = pd.read_csv('./ToDemotestData.csv', delimiter=';')
df.drop(columns='Unnamed: 0', inplace=True)
print("Columns: {}, Shape: {}".format(list(df.columns), df.shape))

Columns: ['People', 'avgSpend'], Shape: (100, 2)


In [14]:
r = DataFrameReader(df)

In [15]:
data = r.read()

2020-02-01 16:20:30,008 - 4506797504 - INFO - read 100
2020-02-01 16:20:30,009 - 4506797504 - INFO - attribute names: [('id', <class 'float'>), ('People', <class 'float'>), ('avgSpend', <class 'float'>)]


In [16]:
min_split = np.ceil(data.getLength()* 0.0001)
cltree = CLTree(data, min_split, './Test/')

In [17]:
cltree.buildTree()

XXXXXXXXXX----------XXXXXXXXXX----------XXXXXXXXXX----------XXXXXXXXXX----------XXXXXXXXXX
This is the starting point!
At previous level(0), the cut happened on Attribute:People, Value: 8.0
At previous level(1), the cut happened on Attribute:avgSpend, Value: 78.0
At previous level(2), the cut happened on Attribute:People, Value: 7.0
At previous level(3), the cut happened on Attribute:People, Value: 6.0
At previous level(4), the cut happened on Attribute:People, Value: 1.0
At previous level(5), the cut happened on Attribute:avgSpend, Value: 40.0
At previous level(6), the cut happened on Attribute:avgSpend, Value: 50.0
At previous level(7), the cut happened on Attribute:avgSpend, Value: 51.0
At previous level(4), the cut happened on Attribute:People, Value: 1.0
At previous level(5), the cut happened on Attribute:avgSpend, Value: 23.0
At previous level(6), the cut happened on Attribute:avgSpend, Value: 21.0
At previous level(5), the cut happened on Attribute:avgSpend, Value: 23.0
At previ

In [18]:
from graphviz import Digraph, nohtml

class Queue:
    def __init__(self):
        self.items = []

    def isEmpty(self):
        return self.items == []

    def enqueue(self, item):
        self.items.insert(0,item)

    def dequeue(self):
        return self.items.pop()

    def size(self):
        return len(self.items)


def TreeVisualization(root, filename):
    node = root
    g = Digraph('g', filename=filename, node_attr={'shape': 'record', 'height': '.1'})
    queue = Queue()
    i = 0
    str1 = node.str2()
    queue.enqueue(('node'+str(i), node))
    nodeVal = '<f0> |<f1> {}|<f2>'.format(str1)
    g.node('node'+str(i), nohtml(nodeVal))
    i+=1
    while not queue.isEmpty(): 
  
            # Dequeue a vertex from  
            # queue and print it 
            keyValue = queue.dequeue()
            key = keyValue[0]
            val = keyValue[1]
            if not val.isLeaf():
                lChild = val.getLeft()
                if not lChild.isLeaf():
                    str1 =lChild.str2()
                else:
                    str1 = lChild.str2()
                    
                queue.enqueue(('node'+str(i), lChild))
                nodeVal = '<f0> |<f1> {}|<f2>'.format(str1)
                g.node('node'+str(i), nohtml(nodeVal))
                g.edge('{}:f0'.format(key), '{}:f1'.format('node'+str(i)))
                i+=1
                if val.getRight():
                    rChild = val.getRight()
                    if not rChild.isLeaf():
                        str2 = rChild.str2()
                    else:
                        str2 = rChild.str2()
                        
                    queue.enqueue(('node'+str(i), rChild))
                    nodeVal = '<f0> |<f1> {}|<f2>'.format(str2)
                    g.node('node'+str(i), nohtml(nodeVal))
                    g.edge('{}:f2'.format(key), '{}:f1'.format('node'+str(i)))
                    i+=1
                
    g.view() 

In [19]:
TreeVisualization(cltree.root, './initialCLTree-TestData-toDemo-Iterative-LeastMemory.gv')

In [18]:
max(cltree.nodes)

166

In [19]:
cltree.getRoot().str2()

'ID: 0 Inst: 100 Depth: 0'

In [20]:
cltree.getRoot().datasetPhysicalAddress

'./Test/CLTree/1580580414/DataSet/DataSet_1580580414.pickle'

In [21]:
cltree.nodes[21].datasetId

'DataSet_19_R'

In [19]:
cltree.nodes[44].datasetSummary.__str__()

"Data Set Id: DataSet_42_R Physical Address: ./Test/CLTree/1580485197/DataSet/DataSet_42_R.pickle Total Number of Data Points: 3 Attributes: ['People', 'avgSpend'] Min Values: [13.  3. 39.] Max Values: [87.  3. 39.]"

In [32]:
cltree.nodes[16].datasetId

'DataSet_12_R'

In [21]:
data = pd.read_csv('datatrain_processed.csv')

In [22]:
data = pd.get_dummies(data)

In [23]:
print("Shape: {}".format(data.shape))

Shape: (1048573, 192)


In [24]:
r = DataFrameReader(data)
data = r.read()

2020-02-01 16:22:51,298 - 4506797504 - INFO - read 1048573
2020-02-01 16:22:51,299 - 4506797504 - INFO - attribute names: [('id', <class 'float'>), ('NumCodeShareSegments', <class 'float'>), ('outb-DeptDOW', <class 'float'>), ('outb-NumStops', <class 'float'>), ('TripOrigCntry_ae', <class 'float'>), ('TripOrigCntry_at', <class 'float'>), ('TripOrigCntry_au', <class 'float'>), ('TripOrigCntry_az', <class 'float'>), ('TripOrigCntry_bd', <class 'float'>), ('TripOrigCntry_be', <class 'float'>), ('TripOrigCntry_bh', <class 'float'>), ('TripOrigCntry_bw', <class 'float'>), ('TripOrigCntry_by', <class 'float'>), ('TripOrigCntry_ca', <class 'float'>), ('TripOrigCntry_ch', <class 'float'>), ('TripOrigCntry_cn', <class 'float'>), ('TripOrigCntry_de', <class 'float'>), ('TripOrigCntry_eg', <class 'float'>), ('TripOrigCntry_es', <class 'float'>), ('TripOrigCntry_fr', <class 'float'>), ('TripOrigCntry_gb', <class 'float'>), ('TripOrigCntry_gr', <class 'float'>), ('TripOrigCntry_hk', <class 'float'>

In [25]:
min_split = np.ceil(data.getLength()* 0.001)
cltree = CLTree(data, min_split, './Test-3/')

In [26]:
import datetime
print (datetime.datetime.now())

2020-02-01 16:23:05.139847


In [27]:
cltree.buildTree()

XXXXXXXXXX----------XXXXXXXXXX----------XXXXXXXXXX----------XXXXXXXXXX----------XXXXXXXXXX
This is the starting point!
At previous level(0), the cut happened on Attribute:TripOrigCntry_sc, Value: 0.0
At previous level(1), the cut happened on Attribute:TripOrigCntry_at, Value: 0.0
At previous level(2), the cut happened on Attribute:TripDestCntry_kz, Value: 0.0
At previous level(3), the cut happened on Attribute:inb-NumStops_3, Value: 0.0
At previous level(4), the cut happened on Attribute:TripOrigCntry_mv, Value: 0.0
At previous level(5), the cut happened on Attribute:TripOrigCntry_az, Value: 0.0
At previous level(6), the cut happened on Attribute:TripDestCntry_by, Value: 0.0
At previous level(7), the cut happened on Attribute:TripOrigCntry_sd, Value: 0.0
At previous level(8), the cut happened on Attribute:Cabin_4 - premium business, Value: 0.0
At previous level(9), the cut happened on Attribute:TripOrigCntry_by, Value: 0.0
At previous level(10), the cut happened on Attribute:TripOrigCn

At previous level(98), the cut happened on Attribute:NumCodeShareSegments, Value: 1.0
At previous level(99), the cut happened on Attribute:TripDestCntry_id, Value: 0.0
At previous level(100), the cut happened on Attribute:AP-BIN_(5.059, 5.565], Value: 0.0
At previous level(101), the cut happened on Attribute:TripOrigCntry_de, Value: 0.0
At previous level(102), the cut happened on Attribute:TripDestCntry_th, Value: 0.0
At previous level(103), the cut happened on Attribute:LOS-BIN_(4.576, 5.23], Value: 0.0
At previous level(104), the cut happened on Attribute:TripOrigCntry_ca, Value: 0.0
At previous level(105), the cut happened on Attribute:AP-BIN_(-0.00607, 0.506], Value: 0.0
At previous level(106), the cut happened on Attribute:NumCodeShareSegments, Value: 0.0
At previous level(107), the cut happened on Attribute:TripOrigCntry_pk, Value: 0.0
At previous level(108), the cut happened on Attribute:TripDestCntry_us, Value: 0.0
At previous level(109), the cut happened on Attribute:NbrInPart

At previous level(148), the cut happened on Attribute:TripOrigCntry_gb, Value: 0.0
At previous level(149), the cut happened on Attribute:outb-DeptDOW, Value: 1.0
At previous level(146), the cut happened on Attribute:SatNightStay_y, Value: 0.0
At previous level(147), the cut happened on Attribute:outb-DeptTimeWindow_00:00_04:59latenight, Value: 0.0
At previous level(145), the cut happened on Attribute:inb-DeptDOW_7, Value: 0.0
At previous level(146), the cut happened on Attribute:TripDestCntry_in, Value: 0.0
At previous level(147), the cut happened on Attribute:outb-DeptTimeWindow_13:00_14:59afternoon, Value: 0.0
At previous level(144), the cut happened on Attribute:LOS-BIN_(2.615, 3.269], Value: 0.0
At previous level(145), the cut happened on Attribute:TripOrigCntry_au, Value: 0.0
At previous level(143), the cut happened on Attribute:inb-segmentMileage-bin_(16.154, 20.193], Value: 0.0
At previous level(144), the cut happened on Attribute:outb-DeptTimeWindow_13:00_14:59afternoon, Value:

At previous level(110), the cut happened on Attribute:NumChildrenBin_0, Value: 0.0
At previous level(111), the cut happened on Attribute:TripDestCntry_ae, Value: 0.0
At previous level(112), the cut happened on Attribute:TripDestCntry_eg, Value: 0.0
At previous level(113), the cut happened on Attribute:inb-DeptTimeWindow_05:00_07:59earlymorning, Value: 0.0
At previous level(114), the cut happened on Attribute:TripOrigCntry_om, Value: 0.0
At previous level(115), the cut happened on Attribute:outb-segmentMileage-bin_(7.995, 10.375], Value: 0.0
At previous level(116), the cut happened on Attribute:inb-segmentMileage-bin_(8.077, 12.116], Value: 0.0
At previous level(117), the cut happened on Attribute:TripOrigCntry_ae, Value: 0.0
At previous level(118), the cut happened on Attribute:LOS-BIN_(1.961, 2.615], Value: 0.0
At previous level(119), the cut happened on Attribute:inb-DeptTimeWindow_11:00_12:59latemorning, Value: 0.0
At previous level(120), the cut happened on Attribute:AP-BIN_(4.553,

At previous level(133), the cut happened on Attribute:SatNightStay_n, Value: 0.0
At previous level(134), the cut happened on Attribute:outb-segmentMileage-bin_(17.514, 19.893], Value: 0.0
At previous level(135), the cut happened on Attribute:TripOrigCntry_gb, Value: 0.0
At previous level(136), the cut happened on Attribute:Cabin_5 - business, Value: 0.0
At previous level(137), the cut happened on Attribute:TripDestCntry_gb, Value: 0.0
At previous level(138), the cut happened on Attribute:TripOrigCntry_us, Value: 0.0
At previous level(139), the cut happened on Attribute:outb-segmentMileage-bin_(19.893, 22.273], Value: 0.0
At previous level(140), the cut happened on Attribute:AP-BIN_(4.047, 4.553], Value: 0.0
At previous level(141), the cut happened on Attribute:TripDestCntry_ph, Value: 0.0
At previous level(142), the cut happened on Attribute:outb-segmentMileage-bin_(15.134, 17.514], Value: 0.0
At previous level(143), the cut happened on Attribute:inb-DeptDOW_2, Value: 0.0
At previous l

...

At previous level(54), the cut happened on Attribute:outb-DeptTimeWindow_21:00_23:59lateevening, Value: 0.0
At previous level(52), the cut happened on Attribute:inb-DeptTimeWindow_15:00_17:59lateafternoon, Value: 0.0
At previous level(53), the cut happened on Attribute:Cabin_5 - business, Value: 0.0
At previous level(51), the cut happened on Attribute:NbrInPartyBin_5plus, Value: 0.0
At previous level(52), the cut happened on Attribute:NumCodeShareSegments, Value: 0.0
At previous level(50), the cut happened on Attribute:outb-segmentMileage-bin_(19.893, 22.273], Value: 0.0
At previous level(51), the cut happened on Attribute:NumCodeShareSegments, Value: 0.0
At previous level(49), the cut happened on Attribute:TripDestCntry_au, Value: 0.0
At previous level(50), the cut happened on Attribute:outb-DeptTimeWindow_13:00_14:59afternoon, Value: 0.0
At previous level(48), the cut happened on Attribute:inb-segmentMileage-bin_(20.193, 24.231], Value: 0.0
At previous level(49), the cut happened on 

At previous level(50), the cut happened on Attribute:Cabin_5 - business, Value: 0.0
At previous level(48), the cut happened on Attribute:AP-BIN_(4.553, 5.059], Value: 0.0
At previous level(49), the cut happened on Attribute:TripDestCntry_ch, Value: 0.0
At previous level(47), the cut happened on Attribute:NbrInPartyBin_4, Value: 0.0
At previous level(48), the cut happened on Attribute:Cabin_5 - business, Value: 0.0
At previous level(46), the cut happened on Attribute:NumChildrenBin_2, Value: 0.0
At previous level(47), the cut happened on Attribute:inb-DeptTimeWindow_15:00_17:59lateafternoon, Value: 0.0
At previous level(45), the cut happened on Attribute:AP-BIN_(-0.00607, 0.506], Value: 0.0
At previous level(46), the cut happened on Attribute:TripDestCntry_ch, Value: 0.0
At previous level(44), the cut happened on Attribute:NbrInPartyBin_5plus, Value: 0.0
At previous level(45), the cut happened on Attribute:TripDestCntry_ch, Value: 0.0
At previous level(43), the cut happened on Attribute

At previous level(40), the cut happened on Attribute:outb-NumStops, Value: 1.0
At previous level(38), the cut happened on Attribute:inb-DeptTimeWindow_11:00_12:59latemorning, Value: 0.0
At previous level(39), the cut happened on Attribute:TripOrigCntry_cn, Value: 0.0
At previous level(37), the cut happened on Attribute:LOS-BIN_(5.23, 5.883], Value: 0.0
At previous level(38), the cut happened on Attribute:Cabin_2 - first, Value: 0.0
At previous level(36), the cut happened on Attribute:NbrInPartyBin_5plus, Value: 0.0
At previous level(37), the cut happened on Attribute:Cabin_5 - business, Value: 0.0
At previous level(35), the cut happened on Attribute:AP-BIN_(5.565, 6.071], Value: 0.0
At previous level(36), the cut happened on Attribute:Cabin_7 - premium economy, Value: 0.0
At previous level(34), the cut happened on Attribute:outb-DeptTimeWindow_13:00_14:59afternoon, Value: 0.0
At previous level(35), the cut happened on Attribute:TripType_ow, Value: 0.0
At previous level(33), the cut hap

At previous level(31), the cut happened on Attribute:NumChildrenBin_3, Value: 0.0
At previous level(32), the cut happened on Attribute:AP-BIN_(-0.00607, 0.506], Value: 0.0
At previous level(33), the cut happened on Attribute:NumCodeShareSegments, Value: 4.0
At previous level(34), the cut happened on Attribute:LOS-BIN_(0.654, 1.307], Value: 0.0
At previous level(35), the cut happened on Attribute:inb-DeptTimeWindow_08:00_10:59morning, Value: 0.0
At previous level(36), the cut happened on Attribute:AP-BIN_(5.565, 6.071], Value: 0.0
At previous level(37), the cut happened on Attribute:outb-segmentMileage-bin_(22.273, 24.653], Value: 0.0
At previous level(38), the cut happened on Attribute:outb-DeptTimeWindow_05:00_07:59earlymorning, Value: 0.0
At previous level(39), the cut happened on Attribute:Cabin_7 - premium economy, Value: 0.0
At previous level(40), the cut happened on Attribute:NumCodeShareSegments, Value: 0.0
At previous level(38), the cut happened on Attribute:outb-DeptTimeWindow

At previous level(16), the cut happened on Attribute:AP-BIN_(4.553, 5.059], Value: 0.0
At previous level(14), the cut happened on Attribute:TripOrigCntry_kz, Value: 0.0
At previous level(15), the cut happened on Attribute:outb-DeptTimeWindow_00:00_04:59latenight, Value: 0.0
At previous level(13), the cut happened on Attribute:outb-NumStops, Value: 2.0
At previous level(14), the cut happened on Attribute:TripOrigCntry_bh, Value: 0.0
At previous level(12), the cut happened on Attribute:TripDestCntry_sd, Value: 0.0
At previous level(13), the cut happened on Attribute:inb-DeptTimeWindow_05:00_07:59earlymorning, Value: 0.0
At previous level(11), the cut happened on Attribute:TripDestCntry_az, Value: 0.0
At previous level(12), the cut happened on Attribute:outb-DeptTimeWindow_00:00_04:59latenight, Value: 0.0
At previous level(10), the cut happened on Attribute:TripOrigCntry_nc, Value: 0.0
At previous level(11), the cut happened on Attribute:inb-DeptTimeWindow_08:00_10:59morning, Value: 0.0
A

In [28]:
print (datetime.datetime.now())

2020-02-03 14:41:26.536302


In [30]:
writePickleFile('./Test-3/Large_CLTree.pickle', cltree)

True